In [103]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *

In [104]:
spark = SparkSession.builder.getOrCreate()

In [105]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [23]:
data = np.arange(1,50)
rdd = sc.parallelize(data)
print(rdd.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


Find the sum, average, maximum, minimum, and count

In [24]:
#Sum
print(rdd.sum())

1225


In [25]:
#avg
rdd.mean()

25.0

In [26]:
#Count
rdd.count()

49

In [28]:
#Min
rdd.min()

1

In [27]:
#Max
rdd.max()

49

Count how many numbers are even vs. odd.

In [41]:
even_n = rdd.filter(lambda x : x%2 == 0 )
odd_n = rdd.filter(lambda x : x%2 != 0 )

print(f"ODD Numbers: {odd_n.count()}, EVEN Numbers: {even_n.count()}")

ODD Numbers: 25, EVEN Numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [42]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [153]:
oldest_person = rdd_people.max(key=lambda x: x[1])
print(oldest_person)


('Khaled', 40)


Compute the average age

In [152]:
total_age = rdd_people.map(lambda x: x[1]).sum()
count_people = rdd_people.count()
average_age = total_age / count_people

print(average_age)


31.666666666666668


Group all the names by their age

In [151]:
grouped_by_age = rdd_people.map(lambda x: (x[1], x[0])).groupByKey()
result = grouped_by_age.mapValues(list).collect()

print(result)


[(40, ['Khaled']), (25, ['Nada ', 'Nada ']), (30, ['Mona']), (35, ['Ahmed', 'Ahmed'])]


Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"

In [54]:
rdd_russia = sc.textFile("/data/russia.txt")
rdd_russia.collect()

['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production']

Count the total number of lines.

In [55]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [124]:
russia_count = rdd_russia.filter(lambda line: 'Russia' in line).count()
print(russia_count)


6


Find the most 5 frequent word in the file.

Tokenize words

In [134]:
words = rdd_russia.flatMap(lambda line: line.lower().split())
print(words.collect())

['russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'moscow', 'is', 'the', 'capital', 'city', 'of', 'russia', 'the', 'russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'the', 'trans-siberian', 'railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'the', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of). 

In [130]:
stopword = ["a", "the", "is", "to", "in", "of"]
words = rdd_russia.flatMap(lambda line: line.lower().split())
without_sw = words.filter(lambda word: word not in stopword)
print(without_sw.collect())

['russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'trans-siberian', 'railway', 'longest', 'railway', 'line', 'world', 'russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [56]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [58]:
df.printSchema()
df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)



+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [63]:
name_sal = df.select("name","salary")
name_sal.show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [70]:
import pyspark.sql.functions as f
avg_sal = df.agg(f.avg(df["salary"]).alias('Avg Salary'))
avg_sal.show()

+----------+
|Avg Salary|
+----------+
|    6000.0|
+----------+



Filter employees older than 28

In [79]:
emp_older = df.filter(df["age"] > 28)
emp_older.show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [97]:
distinct = df.select("name").distinct().count()
distinct

4

Group by a the name column and find average salary

In [107]:
name_avg = df.groupBy('name').agg(f.avg(df['salary']).alias('Average'))
name_avg.show()

+------+-------+
|  name|Average|
+------+-------+
|   Ali| 4000.0|
|  Omar| 6750.0|
|Mariam| 6750.0|
|  Sara| 5000.0|
+------+-------+



In [106]:
df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|  345|
|emp4|Cindy|  456|
+----+-----+-----+



Find the avg sales 

In [110]:
Avg_sales = df1.select(f.avg('Sales'))
average = Avg_sales.collect()[0][0]
average

400.5

Replace null name with 'Unknown' and sales with the avg sales of the column 

In [111]:
df1.fillna(
    {
        'Name' : "Unknown",
        'Sales' : average
    }
).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|  400|
|emp2|Unknown|  400|
|emp3|Unknown|  345|
|emp4|  Cindy|  456|
+----+-------+-----+

